#### LLM Powered Chatbot

In [5]:

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("QROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model.invoke("Hey").content

'Hey there! 👋 What can I do for you today? 😊\n'

In [12]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hey, my name is Jim and I am a Generative AI Engineer")]).content

"Hi Jim, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of generative AI work are you involved with?  \n\nDo you have any projects you're particularly excited about?\n"

In [14]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content = "Hey, my name is Jim and I am a Generative AI Engineer"),
        AIMessage(content = "Hi Jim, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of generative AI work are you involved with?  \n\nDo you have any projects you're particularly excited about?\n"),
        HumanMessage(content = "What is my name and what do I do ?")
    ]
).content

"You told me your name is Jim and that you are a Generative AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself? 😊  \n"

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat1"}}

In [21]:
response = with_message_history.invoke(
  [HumanMessage(content="Hey my name is Jim and I am an AI Enginner.")],
  config=config
 )

In [22]:
response.content

"Hi Jim! It's great to meet another AI engineer.  \n\nWhat brings you here today? Are you working on any interesting projects?  Perhaps I could be of assistance? \n\n"

In [25]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
).content

"You said your name is Jim when you first introduced yourself. 😊 \n\nIs there something specific you'd like to know about your name? For example, are you curious about its origin or meaning?  \n\n\n\n\n\n"

In [27]:
#chamging the config
config_1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name")],
    config=config_1
)
print(response.content)

As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'm happy to use it!



In [28]:
response = with_message_history.invoke(
    [HumanMessage(content = "My name is jimmy")],
    config=config_1
)
print(response.content)

Hi Jimmy! It's nice to meet you. 😊

Is there anything I can help you with today?



In [29]:
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name")],
    config=config_1
)
print(response.content)

Your name is Jimmy!  

I remember you told me a moment ago. 😊  Anything else I can help you with, Jimmy?



### Prompt Templates

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [35]:
chain.invoke({"messages": [HumanMessage(content="Hey my name is Jim")]}).content

"Hi Jim, it's nice to meet you!  \n\nWhat can I do for you today? 😊  \n\n"

In [36]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [38]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Kamau")],
    config=config
)
response

AIMessage(content="Hello Kamau, it's nice to meet you! 👋 \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 32, 'total_tokens': 60, 'completion_time': 0.050909091, 'prompt_time': 0.002323441, 'queue_time': 0.186606381, 'total_time': 0.053232532}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--48271a42-f3d5-4668-856b-509dd470abe6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 28, 'total_tokens': 60})

In [39]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [40]:
response = chain.invoke({"messages": [HumanMessage(content="Hi my name is Jim")], "language": "Swahili"})
response.content

'Habari Jim, ni furaha kukutana nawe! Unaweza kuniuliza chochote, nitajaribu kukupa jibu bora iwezekanavyo.  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [49]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/home/brian/Documents/JIM/Gen AI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [51]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor!  \n\nWhat's your favorite flavor? 😊\n"

In [52]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".  😊  \n'

In [53]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [54]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to any past conversations or your personal information. So, I don't know your name.  😊 \n\nWould you like to tell me your name?\n"

In [55]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem now, I'd be happy to help! 😊  \n\n"